In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

REPO_ROOT = Path.cwd().parent  # running inside /notebooks
OUT_ROOT = REPO_ROOT / "outputs" / "rtm"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

print("Repo root:", REPO_ROOT)
print("Out root:", OUT_ROOT)

Repo root: c:\Users\C.Price\Habnetic\resilient-housing-bayes
Out root: c:\Users\C.Price\Habnetic\resilient-housing-bayes\outputs\rtm


In [2]:
exp = pd.read_parquet(OUT_ROOT / "water_exposure_Ehat_v0.parquet")
haz = pd.read_parquet(OUT_ROOT / "hazard_pluvial_v0.parquet")

df = exp.merge(haz, on="bldg_id", how="inner")
assert len(df) == len(exp), "Join mismatch: exposure vs hazard"
df.head()


,bldg_id,E_hat,z_d,z_250,z_500,z_1000,H_pluvial_v0
0,305012,0.536838,0.963020,0.802751,0.373586,0.007996,1.0
1,313960,0.677579,1.383126,0.917711,0.437259,-0.027780,1.0
2,313263,0.251841,-0.214838,0.809959,0.418931,-0.006689,1.0
3,310491,0.189019,-0.294736,0.644668,0.391794,0.014349,1.0
4,313127,-0.292821,-0.311231,-1.268695,0.414013,-0.005372,1.0


In [3]:
rng = np.random.default_rng(42)

# Simple logistic link: p(damage) = sigmoid(alpha + beta*E_hat + gamma*H)
alpha = -2.0   # baseline rarity
beta  =  1.0   # exposure effect
gamma =  0.0   # hazard effect (0 for now because H is constant placeholder)

lin = alpha + beta * df["E_hat"].to_numpy() + gamma * df["H_pluvial_v0"].to_numpy()
p = 1.0 / (1.0 + np.exp(-lin))

df["Y_damage_v0"] = rng.binomial(n=1, p=p).astype("int8")

df[["bldg_id", "E_hat", "H_pluvial_v0", "Y_damage_v0"]].head(), df["Y_damage_v0"].mean()


(   bldg_id     E_hat  H_pluvial_v0  Y_damage_v0
 0   305012  0.536838           1.0            0
 1   313960  0.677579           1.0            0
 2   313263  0.251841           1.0            1
 3   310491  0.189019           1.0            0
 4   313127 -0.292821           1.0            0,
 0.1429487990457429)

In [4]:
outcome = df[["bldg_id", "Y_damage_v0"]].copy()

out_path = OUT_ROOT / "outcome_damage_v0.parquet"
outcome.to_parquet(out_path, index=False)

print("Wrote:", out_path)
print("Damage rate:", float(outcome["Y_damage_v0"].mean()))


Wrote: c:\Users\C.Price\Habnetic\resilient-housing-bayes\outputs\rtm\outcome_damage_v0.parquet
Damage rate: 0.1429487990457429


In [5]:
# Re-load and re-join everything to ensure artifacts are consistent
out = pd.read_parquet(out_path)
df2 = exp.merge(haz, on="bldg_id").merge(out, on="bldg_id")

assert len(df2) == len(exp)
assert df2["Y_damage_v0"].isin([0, 1]).all()
df2.head()


,bldg_id,E_hat,z_d,z_250,z_500,z_1000,H_pluvial_v0,Y_damage_v0
0,305012,0.536838,0.963020,0.802751,0.373586,0.007996,1.0,0
1,313960,0.677579,1.383126,0.917711,0.437259,-0.027780,1.0,0
2,313263,0.251841,-0.214838,0.809959,0.418931,-0.006689,1.0,1
3,310491,0.189019,-0.294736,0.644668,0.391794,0.014349,1.0,0
4,313127,-0.292821,-0.311231,-1.268695,0.414013,-0.005372,1.0,0


In [7]:
import json

meta = {
    "schema_version": "0.1.0",
    "outcome": "Y_damage_v0",
    "type": "synthetic",
    "definition": "Bernoulli with p = sigmoid(alpha + beta*E_hat + gamma*H_pluvial_v0)",
    "params": {"alpha": alpha, "beta": beta, "gamma": gamma, "seed": 42},
    "constraints": [
        "Synthetic placeholder outcome for pipeline wiring.",
        "Not calibrated to observed damages.",
        "Do not interpret as real flood damage probability."
    ],
}

meta_path = OUT_ROOT / "outcome_damage_v0_meta.json"
meta_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")
print("Wrote:", meta_path)


Wrote: c:\Users\C.Price\Habnetic\resilient-housing-bayes\outputs\rtm\outcome_damage_v0_meta.json


In [8]:
outcome_rate = df["Y_damage_v0"].mean()
outcome_rate


0.1429487990457429

In [9]:
df.groupby(pd.qcut(df["E_hat"], 10))["Y_damage_v0"].mean()


E_hat
(-2.893, -1.326]    0.025392
(-1.326, -0.517]    0.056163
(-0.517, -0.31]     0.088017
(-0.31, 0.106]      0.101342
(0.106, 0.313]      0.144180
(0.313, 0.418]      0.162344
(0.418, 0.511]      0.174129
(0.511, 0.62]       0.192527
(0.62, 0.776]       0.212769
(0.776, 2.268]      0.272625
Name: Y_damage_v0, dtype: float64